In [1]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from tqdm import tqdm
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
dataset_name = "MelodyOfTears/sst2-with-cot"
enhanced_dataset = load_dataset(dataset_name, split="train[:500]")

README.md:   0%|          | 0.00/453 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/98.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [4]:
# 初始化模型
max_seq_length = 2048
dtype = torch.float16  # 用于4位量化
load_in_4bit = True

print("正在加载模型...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-7B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

正在加载模型...
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [5]:
# 定义训练用的提示模板
train_prompt_style = """Below is an instruction that describes a task, paired with
an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by
step chain of thoughts to ensure a logical and accurate response.
### Instruction:
You are an expert in sentiment analysis with advanced knowledge in understanding
and interpreting emotions from text.
Please analyze the sentiment of the following text and output 0 (negative) or 1 (positive).
### Text:
{}
### Response:
<think>
{}
</think>
{}"""

# 格式化数据集用于训练
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    inputs = examples["sentence"]
    cots = examples["Complex_CoT"]
    outputs = examples["label"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

formatted_dataset = enhanced_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [6]:
# 设置LoRA用于微调
print("\n正在设置LoRA用于微调...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# 设置训练器
print("正在设置训练器...")
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        num_train_epochs=1,
        warmup_ratio=0.1,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none"  # 禁用wandb
    ),
)


正在设置LoRA用于微调...


Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


正在设置训练器...


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

In [7]:
prompt_style = """Below is an instruction that describes a task, paired with
an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by
step chain of thoughts to ensure a logical and accurate response.
### Instruction:
You are an expert in sentiment analysis with advanced knowledge in understanding
and interpreting emotions from text.
Please analyze the sentiment of the following text and output 0 (negative) or 1 (positive).
### Text:
{}
### Response:
<think>{}"""

In [8]:
def inference_example(text):
    FastLanguageModel.for_inference(model)
    inputs = tokenizer([prompt_style.format(text, "")], return_tensors="pt").to("cuda")
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=1200,
        use_cache=True,
    )
    response = tokenizer.batch_decode(outputs)
    return response[0].split("### Response:")[1]

In [9]:
def evaluate_model_on_sst2():
    validation_dataset = load_dataset("glue", "sst2", split="validation")

    FastLanguageModel.for_inference(model)

    correct_predictions = 0
    total_predictions = 0

    progress_bar = tqdm(validation_dataset, desc="评估SST2验证集")

    for sample in progress_bar:
        text = sample["sentence"]
        true_label = sample["label"]

        try:
            
            inputs = tokenizer([prompt_style.format(text, "")], return_tensors="pt").to("cuda")
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=300,  
                use_cache=True,
            )
                        
            response_after_think = tokenizer.batch_decode(outputs)[0].split("</think>")[1].strip()
            pred_label_match = re.search(r'[01]', response_after_think)

            if pred_label_match:
                pred_label = int(pred_label_match.group())

                if pred_label == true_label:
                    correct_predictions += 1
                total_predictions += 1

                current_accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
                progress_bar.set_description(f"准确率: {current_accuracy:.4f}")
        except Exception as e:
            print(f"处理样本时出错: {e}")

    # 计算最终准确率
    final_accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    print(f"\nSST2验证集上的评估结果:")
    print(f"评估的样本总数: {total_predictions}")
    print(f"正确预测数: {correct_predictions}")
    print(f"准确率: {final_accuracy:.4f}")

    return final_accuracy

In [10]:
accuracy = evaluate_model_on_sst2()

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

...

处理样本时出错: list index out of range


准确率: 0.8860: 100%|█████████▉| 870/872 [2:57:00<00:25, 12.65s/it]

处理样本时出错: list index out of range


准确率: 0.8864: 100%|██████████| 872/872 [2:57:24<00:00, 12.21s/it]


SST2验证集上的评估结果:
评估的样本总数: 537
正确预测数: 476
准确率: 0.8864
